In [2]:
# Import necessary packages
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
import pickle
import networkx as nx
import random as rand
import matplotlib.pyplot as plt
from qiskit import Aer, execute
from scipy.optimize import minimize
#way to visualize the graphs
def DrawGraph(Graph):
    plt.rcParams["figure.figsize"] = (10,6)
    pos=nx.spring_layout(Graph) 
    nx.draw_networkx(Graph,pos)
    labels = nx.get_edge_attributes(Graph,'weight')
    nx.draw_networkx_edge_labels(Graph,pos,edge_labels=labels)
    plt.show()
    return 
#way to convert the graphs to a easy formatable manner
def matrix_to_edges(matrix):
    edges = []
    num_rows, num_cols = matrix.shape

    for row in range(num_rows):
        for col in range(row,num_cols):
            if matrix[row, col] != 0 and row!=col:
                edges.append((row, col))

    return edges

from qiskit import QuantumCircuit, Aer
from qiskit.circuit import Parameter
def generate_binary_strings():
    binary_strings = []
    for i in range(256):
        binary_string = format(i, '08b')
        binary_strings.append(binary_string)
    return binary_strings
binary_strings = generate_binary_strings()
print(binary_strings)
def order_dictionary(dictionary):
    ordered_dict = {}
    sorted_keys = binary_strings  # Sort the keys in ascending order
    for key in sorted_keys:
        ordered_dict[key] = dictionary.get(key, 0)  # Add key-value pairs to the ordered dictionary
    dict_jdd = {}
    for i in range(128):
        dict_jdd[binary_strings[i]]=ordered_dict[binary_strings[i]] + ordered_dict[binary_strings[255-i]]
    return dict_jdd

['00000000', '00000001', '00000010', '00000011', '00000100', '00000101', '00000110', '00000111', '00001000', '00001001', '00001010', '00001011', '00001100', '00001101', '00001110', '00001111', '00010000', '00010001', '00010010', '00010011', '00010100', '00010101', '00010110', '00010111', '00011000', '00011001', '00011010', '00011011', '00011100', '00011101', '00011110', '00011111', '00100000', '00100001', '00100010', '00100011', '00100100', '00100101', '00100110', '00100111', '00101000', '00101001', '00101010', '00101011', '00101100', '00101101', '00101110', '00101111', '00110000', '00110001', '00110010', '00110011', '00110100', '00110101', '00110110', '00110111', '00111000', '00111001', '00111010', '00111011', '00111100', '00111101', '00111110', '00111111', '01000000', '01000001', '01000010', '01000011', '01000100', '01000101', '01000110', '01000111', '01001000', '01001001', '01001010', '01001011', '01001100', '01001101', '01001110', '01001111', '01010000', '01010001', '01010010', '01

In [6]:
#here I am going to import the data that we set for the NN network
# Specify the file path of the pickle file
file_path = r'C:\Users\evilb\OneDrive - University of Tennessee\Research_Work\QAOA\QAOACountsJuly7_8.pkl'

# Open the pickle file in binary mode and load the data using pickle.load()
with open(file_path, 'rb') as file:
    data_dict = pickle.load(file)

# Now you can use the loaded dictionary
print(len(data_dict['graphs']))
from torchvision import datasets, transforms
import itertools
array = torch.arange(12346)
# Get the size of the array
n = array.size(0)
# Generate a random permutation
torch.manual_seed(42)
perm = torch.randperm(n)
# Use the permutation to randomly organize the array
random_array = array[perm]

traina = []
vala=[]
valnums=[]
trainnums = []
valclock=0
for i in range(0,9877):
    #pay close attendtion to how I fill traina and vala. I call for traina[0] for the counts and so on.
    traina.append([list(data_dict['counts'][random_array[i]].values())])
    traina[i].append(list(data_dict['obj_sol'][random_array[i]]))
    traina[i].append(list(data_dict['binary_sol'][random_array[i]]))
    traina[i].append(list(data_dict['graphs'][random_array[i]]))
    trainnums.append(i)
    #try to append it to the array : trainset[i].append
for i in range(9877,12346):
    vala.append([list(data_dict['counts'][random_array[i]].values())])
    vala[valclock].append(list(data_dict['obj_sol'][random_array[i]]))
    vala[valclock].append(list(data_dict['binary_sol'][random_array[i]]))
    vala[valclock].append(list(data_dict['graphs'][random_array[i]]))
    valnums.append(valclock)
    valclock+=1
trainset = torch.tensor(trainnums)
valset = torch.tensor(valnums)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=256, shuffle=True)


12346


Here I make simple fully connected models to test the legitamacy of training a NN to predict the maximum cut and optimal solution from just the distribution, the graph, or both.

In [137]:
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleDistToOptSol(nn.Module):
    def __init__(self):
        super(SimpleDistToOptSol, self).__init__()
        self.hidden1 = nn.Linear(128, 64)
        self.hidden2 = nn.Linear(64, 32)
        self.hidden3 = nn.Linear(32, 16)
        self.hidden4 = nn.Linear(16, 16)
        self.output = nn.Linear(16, 8)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = torch.relu(self.hidden3(x))
        x = torch.relu(self.hidden4(x))
        x = self.output(x)
        x = self.sigmoid(x)
        return x
# Create an instance of the neural network
net1 = SimpleDistToOptSol()
# Print the architecture of the network
print(net1)

class SimpleGraphToOptSol(nn.Module):
    def __init__(self):
        super(SimpleGraphToOptSol, self).__init__()
        self.hidden1 = nn.Linear(64, 64)
        self.hidden2 = nn.Linear(64, 32)
        self.hidden3 = nn.Linear(32, 16)
        self.hidden4 = nn.Linear(16, 16)
        self.output = nn.Linear(16, 8)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.to(self.hidden1.weight.dtype)
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = torch.relu(self.hidden3(x))
        x = torch.relu(self.hidden4(x))
        x = self.output(x)
        x = self.sigmoid(x)
        return x
# Create an instance of the neural network
net2 = SimpleGraphToOptSol()
# Print the architecture of the network
print(net2)

#this next model is the last two combined.
class SimpleGraphDistToOptSol(nn.Module):
    def __init__(self):
        super(SimpleGraphDistToOptSol, self).__init__()
        self.hidden1 = nn.Linear(64, 64)
        self.hidden2 = nn.Linear(64, 32)
        self.hidden3 = nn.Linear(32, 16)
        self.hidden4 = nn.Linear(16, 16)
        self.output = nn.Linear(16, 16)
        self.sigmoid = nn.Sigmoid()
        self.hidden5 = nn.Linear(128, 64)
        self.hidden6 = nn.Linear(64, 32)
        self.hidden7 = nn.Linear(32, 16)
        self.hidden8 = nn.Linear(16, 16)
        self.output = nn.Linear(16, 16)
        self.sigmoid = nn.Sigmoid()
        self.hidden9 = nn.Linear(32,32)
        self.hidden10 = nn.Linear(32,16)
        self.hidden11 = nn.Linear(16,8)

    def forward(self, x, x1):
        x = x.to(self.hidden1.weight.dtype)
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = torch.relu(self.hidden3(x))
        x = torch.relu(self.hidden4(x))
        x = self.output(x)
        x1 = torch.relu(self.hidden5(x1))
        x1 = torch.relu(self.hidden6(x1))
        x1 = torch.relu(self.hidden7(x1))
        x1 = torch.relu(self.hidden8(x1))
        x1 = self.output(x1)
        x = torch.cat((x1, x), dim=1)
        x = torch.relu(self.hidden9(x))
        x = torch.relu(self.hidden10(x))
        x = torch.relu(self.hidden11(x))
        x = self.sigmoid(x)
        return x
# Create an instance of the neural network
net3 = SimpleGraphDistToOptSol()
# Print the architecture of the network
print(net3)

SimpleDistToOptSol(
  (hidden1): Linear(in_features=128, out_features=64, bias=True)
  (hidden2): Linear(in_features=64, out_features=32, bias=True)
  (hidden3): Linear(in_features=32, out_features=16, bias=True)
  (hidden4): Linear(in_features=16, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=8, bias=True)
  (sigmoid): Sigmoid()
)
SimpleGraphToOptSol(
  (hidden1): Linear(in_features=64, out_features=64, bias=True)
  (hidden2): Linear(in_features=64, out_features=32, bias=True)
  (hidden3): Linear(in_features=32, out_features=16, bias=True)
  (hidden4): Linear(in_features=16, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=8, bias=True)
  (sigmoid): Sigmoid()
)
SimpleGraphDistToOptSol(
  (hidden1): Linear(in_features=64, out_features=64, bias=True)
  (hidden2): Linear(in_features=64, out_features=32, bias=True)
  (hidden3): Linear(in_features=32, out_features=16, bias=True)
  (hidden4): Linear(in_features=16, out_features=16, bi

In [143]:
def train_graph_opt(model,optimizer,loss_fn,train_loader, val_loader,epochs,dataset,number1,number2):
    #time0 = time()
    device = "cpu"
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(device)
    for epoch in range(epochs):
        running_loss = 0
        model.train()
        model.to(device)
        loss_fn.to(device)
        for ticks in train_loader:
            histo=[]
            gatrix=[]
            for i in range(0,len(ticks)):
                histo.append(traina[ticks[i]][number1])
                gatrix.append(traina[ticks[i]][number2])
            gatrix = [[int(bit) for bit in binary_string] for binary_string in gatrix]
            inputs = torch.squeeze(torch.tensor(histo), dim=1).to(device)
            target = torch.squeeze(torch.tensor(gatrix), dim=1).type(torch.float).to(device)
            #print(inputs.shape)

        
            # Training pass
            optimizer.zero_grad()
            #inputs = inputs.requires_grad_()  # Enable gradient tracking
            output = model(inputs)
            #print(output.squeeze().shape, target.shape)
            loss1 = loss_fn(output.squeeze(), target)
            loss=loss1
            #loss.requires_grad=True
            #a=list(model.parameters())[0].clone()
            loss.backward()
            optimizer.step()
            #b=list(model.parameters())[0].clone()
            #print(torch.equal(a.data,b.data))
            running_loss += loss.item()
        #training_loss /= len(train_loader)
        else:
            print("Epoch {} - Training loss: {}".format(epoch, running_loss/len(trainloader)))
        test_graph_sol(model,optimizer,loss_fn, val_loader,dataset,number1,number2)
        test_graph_sol(model,optimizer,loss_fn, train_loader,traina,number1,number2)

def test_graph_sol(model,optimizer,loss_fn, val_loader,dataset,number1,number2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    num_correct =0
    num=0
    num_correct_graphs=0
    model.to(device)
    for valticks in val_loader:
        valhisto=[]
        valgatrix=[]
        for i in range(0,len(valticks)):
            valhisto.append(dataset[valticks[i]][number1])
            valgatrix.append(dataset[valticks[i]][number2])
        valgatrix = [[int(bit) for bit in binary_string] for binary_string in valgatrix]
        inputs = (torch.squeeze(torch.tensor(valhisto), dim=1)).to(device)
        targets = (torch.squeeze(torch.tensor(valgatrix), dim=0).type(torch.float)).to(device)
        output =(model(inputs) > 0.5).float().to(device).squeeze()
        #print(output.shape, targets.shape)
        #print(output)

        correct = torch.eq(output,targets)
        num +=correct.shape[0]
        #print(output,"-",targets)
        #print(correct)
        correct = torch.eq(output,targets)
        num_correct +=torch.sum(correct).item()
        correct_graphs = torch.all(correct, dim=1, keepdim=True).view(-1)
        num_correct_graphs += torch.sum(correct_graphs).item()
        #print(num_correct_graphs)
    print('accuracy: = {:.2f}, number of graphs: = {:.2f}'.format((num_correct_graphs) / (num),num))
        #print(len(output))
        #print(len(targets))


#----------------------------------
#-----------------------------------
def train_graph_opt2(model,optimizer,loss_fn,train_loader, val_loader,epochs,dataset,number2):
    #time0 = time()
    device = "cpu"
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(device)
    for epoch in range(epochs):
        running_loss = 0
        model.train()
        model.to(device)
        loss_fn.to(device)
        for ticks in train_loader:
            histo1=[]
            histo2=[]
            gatrix=[]
            for i in range(0,len(ticks)):
                histo1.append(traina[ticks[i]][3])
                histo2.append(traina[ticks[i]][0])
                gatrix.append(traina[ticks[i]][number2])
            gatrix = [[int(bit) for bit in binary_string] for binary_string in gatrix]
            inputs1 = torch.squeeze(torch.tensor(histo1), dim=1).to(device)
            inputs2 = torch.squeeze(torch.tensor(histo2), dim=1).to(device)

            target = torch.squeeze(torch.tensor(gatrix), dim=1).type(torch.float).to(device)
            #print(inputs.shape)

        
            # Training pass
            optimizer.zero_grad()
            #inputs = inputs.requires_grad_()  # Enable gradient tracking
            output = model(inputs1,inputs2)
            #print(output.squeeze().shape, target.shape)
            loss1 = loss_fn(output.squeeze(), target)
            loss=loss1
            #loss.requires_grad=True
            #a=list(model.parameters())[0].clone()
            loss.backward()
            optimizer.step()
            #b=list(model.parameters())[0].clone()
            #print(torch.equal(a.data,b.data))
            running_loss += loss.item()
        #training_loss /= len(train_loader)
        else:
            print("Epoch {} - Training loss: {}".format(epoch, running_loss/len(trainloader)))
        test_graph_sol2(model,optimizer,loss_fn, val_loader,dataset,number2)
        test_graph_sol2(model,optimizer,loss_fn, train_loader,traina,number2)

def test_graph_sol2(model,optimizer,loss_fn, val_loader,dataset,number2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    num_correct =0
    num=0
    num_correct_graphs=0
    model.to(device)
    for valticks in val_loader:
        valhisto1=[]
        valhisto2=[]
        valgatrix=[]
        for i in range(0,len(valticks)):
            valhisto1.append(dataset[valticks[i]][3])
            valhisto2.append(dataset[valticks[i]][0])
            valgatrix.append(dataset[valticks[i]][number2])
        valgatrix = [[int(bit) for bit in binary_string] for binary_string in valgatrix]
        inputs1 = (torch.squeeze(torch.tensor(valhisto1), dim=1)).to(device)
        inputs2 = (torch.squeeze(torch.tensor(valhisto2), dim=1)).to(device)
        targets = (torch.squeeze(torch.tensor(valgatrix), dim=0).type(torch.float)).to(device)
        output =(model(inputs1,inputs2) > 0.5).float().to(device).squeeze()
        #print(output.shape, targets.shape)
        #print(output)

        correct = torch.eq(output,targets)
        num +=correct.shape[0]
        #print(output,"-",targets)
        #print(output.shape,targets.shape)
        #print(correct)
        correct = torch.eq(output,targets)
        num_correct +=torch.sum(correct).item()
        correct_graphs = torch.all(correct, dim=1, keepdim=True).view(-1)
        num_correct_graphs += torch.sum(correct_graphs).item()
        #print(num_correct_graphs)
    print('accuracy: = {:.2f}, number of graphs: = {:.2f}'.format((num_correct_graphs) / (num),num))
        #print(len(output))
        #print(len(targets))

In [124]:
#train(hcnet,optim.Adam(hcnet.parameters(), lr=.01), torch.nn.CrossEntropyLoss(),trainloader,valloader, 1000,vala)
train_graph_opt(net2,optim.Adam(net2.parameters(), lr=.1), nn.BCELoss(),trainloader,valloader, 1000,vala,3,2)

cpu
Epoch 0 - Training loss: 0.7140062436079367
accuracy: = 0.08, number of graphs: = 2469.00
accuracy: = 0.08, number of graphs: = 9877.00
Epoch 1 - Training loss: 0.6711128002557999
accuracy: = 0.08, number of graphs: = 2469.00
accuracy: = 0.08, number of graphs: = 9877.00


KeyboardInterrupt: 

In [145]:
train_graph_opt(net1,optim.Adam(net1.parameters(), lr=.01), nn.BCELoss(),trainloader,valloader, 1000,vala,0,1)

cpu
Epoch 0 - Training loss: 0.4122377557632251
accuracy: = 0.21, number of graphs: = 2469.00
accuracy: = 0.21, number of graphs: = 9877.00
Epoch 1 - Training loss: 0.2875559123662802
accuracy: = 0.21, number of graphs: = 2469.00
accuracy: = 0.21, number of graphs: = 9877.00
Epoch 2 - Training loss: 0.2871805926163991
accuracy: = 0.21, number of graphs: = 2469.00
accuracy: = 0.21, number of graphs: = 9877.00
Epoch 3 - Training loss: 0.28612974973825306
accuracy: = 0.18, number of graphs: = 2469.00
accuracy: = 0.19, number of graphs: = 9877.00
Epoch 4 - Training loss: 0.2861817631966028
accuracy: = 0.07, number of graphs: = 2469.00
accuracy: = 0.06, number of graphs: = 9877.00
Epoch 5 - Training loss: 0.2854796533401196
accuracy: = 0.18, number of graphs: = 2469.00
accuracy: = 0.18, number of graphs: = 9877.00
Epoch 6 - Training loss: 0.2844198506612044
accuracy: = 0.12, number of graphs: = 2469.00
accuracy: = 0.12, number of graphs: = 9877.00
Epoch 7 - Training loss: 0.2838572286642514

KeyboardInterrupt: 

In [144]:
train_graph_opt2(net3,optim.Adam(net3.parameters(), lr=.01), nn.BCELoss(),trainloader,valloader, 1000,vala,1)

cpu
Epoch 0 - Training loss: 0.6692572832107544
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([165, 8]) torch.Size([165, 8])
accuracy: = 0.00, number of graphs: = 2469.00
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) torch.Size([256, 8])
torch.Size([256, 8]) tor

KeyboardInterrupt: 